# Contours

In [5]:
import cv2
import numpy as np

# Let's load a simple image with 3 black squares
image = cv2.imread('./images/shapes.jpg')
cv2.imshow('Input Image', image)
cv2.waitKey()

# Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Find Canny edges
edged = cv2.Canny(gray, 30, 200)
cv2.imshow('Canny edges', edged)

# Finding Contours
# Use a copy of your image e.g. edged.copy(), since findContours alters the image
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.imshow('Canny Edges After Contouring', edged)
cv2.waitKey(0)

print('Number of contours found = '+ str(len(contours)))

# Draw all contours
# Use '-1' as the 3rd parameter to draw all
# 4th parameter is the colour of the contour
# 5th parameter is the thickness
cv2.drawContours(image, contours, -1, (0,255,0), 3)

cv2.imshow('Contours', image)
cv2.waitKey()
cv2.destroyAllWindows()

Number of contours found = 3


In [7]:
print(hierarchy)

[[[ 1 -1 -1 -1]
  [ 2  0 -1 -1]
  [-1  1 -1 -1]]]


**cv2.findContours(image, Retrieval Mode, Approximation Method)**

Returns -> ret, contours, hierarchy

## NOTE
In OpenCV 2.X, findContours returns only 2 arguments which is contours, hierarchy

If you're using OpenCV 2.X replace line 12 with:

contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

The variable 'contours' are stored as a numpy array of (x,y) points that form the contour

While, 'hierarchy' describes the child-parent relationships between contours (i.e. contours within contours)


## Approximation Methods

Using cv2.CHAIN_APPROX_NONE stores all the boundary points. But we don't necessarily need all bounding points. If the points form a straight line, we only need the start and ending points of that line.

Using cv2.CHAIN_APPROX_SIMPLE instead only provides these start and end points of bounding contours, thus resulting in much more efficent storage of contour information..

### Sorting Contours

We can sort contours in many ways.

In [1]:
import cv2
import numpy as np

# Load our image
image = cv2.imread('images/bunchofshapes.jpg')
cv2.imshow('0 - Original Image', image)
cv2.waitKey(0)

# Create a black image with same dimensions as our loaded image
blank_image = np.zeros((image.shape[0], image.shape[1], 3))

# Create a copy of our original image
orginal_image = image

# Grayscale our image
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# Find Canny edges
edged = cv2.Canny(gray, 50, 200)
cv2.imshow('1 - Canny Edges', edged)
cv2.waitKey(0)

# Find contours and print how many were found
contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print ("Number of contours found = ", len(contours))

#Draw all contours
cv2.drawContours(blank_image, contours, -1, (0,255,0), 3)
cv2.imshow('2 - All Contours over blank image', blank_image)
cv2.waitKey(0)

# Draw all contours over blank image
cv2.drawContours(image, contours, -1, (0,255,0), 3)
cv2.imshow('3 - All Contours', image)
cv2.waitKey(0)

cv2.destroyAllWindows()

Number of contours found =  4


### Let's now sort by area

In [2]:
import cv2
import numpy as np

# Function to desplay contour area

def get_contour_areas(contours):
    # returns the areas of all contours as a list
    all_areas = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        all_areas.append(area)
    return all_areas

# Load our image
image = cv2.imread('images/bunchofshapes.jpg')
original_img = image

# Let's print the areas of the contours before sorting
print('Contour areas before sorting')

# Find contours and print how many were found
contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

print(get_contour_areas(contours))

# Sort contours large to small
sorted_contours = sorted(contours, key = cv2.contourArea, reverse = True)

print('Contour areas after sorting')
print(get_contour_areas(sorted_contours))

# Iterate over our contours and draw one at a time

for c in sorted_contours:
    cv2.drawContours(original_img, [c], -1, (255,0,0), 3)
    cv2.waitKey(0)
    cv2.imshow('Contours by area', original_img)
    
cv2.waitKey()
cv2.destroyAllWindows()

Contour areas before sorting
[20587.5, 22900.5, 66581.5, 90222.0]
Contour areas after sorting
[90222.0, 66581.5, 22900.5, 20587.5]


In [3]:

# Find contours and print how many were found
contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

import cv2
import numpy as np

# Functions we'll use for sorting by position

def x_cord_contour(contours):
    #Returns the X cordinate for the contour centroid
    if cv2.contourArea(contours) > 10:
        M = cv2.moments(contours)
        return (int(M['m10']/M['m00']))
    else:
        pass

    
def label_contour_center(image, c):
    # Places a red circle on the centers of contours
    M = cv2.moments(c)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
 
    # Draw the countour number on the image
    cv2.circle(image,(cx,cy), 10, (0,0,255), -1)
    return image


# Load our image
image = cv2.imread('images/bunchofshapes.jpg')
orginal_image = image.copy()


# Computer Center of Mass or centroids and draw them on our image
for (i, c) in enumerate(contours):
    orig = label_contour_center(image, c)
 
cv2.imshow("4 - Contour Centers ", image)
cv2.waitKey(0)

# Sort by left to right using our x_cord_contour function
contours_left_to_right = sorted(contours, key = x_cord_contour, reverse = False)


# Labeling Contours left to right
for (i,c)  in enumerate(contours_left_to_right):
    cv2.drawContours(orginal_image, [c], -1, (0,0,255), 3)  
    M = cv2.moments(c)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    cv2.putText(orginal_image, str(i+1), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('6 - Left to Right Contour', orginal_image)
    cv2.waitKey(0)
    (x, y, w, h) = cv2.boundingRect(c)  
    
    # Let's now crop each contour and save these images
    cropped_contour = orginal_image[y:y + h, x:x + w]
    image_name = "output_shape_number_" + str(i+1) + ".jpg"
    print(image_name)
    cv2.imwrite(image_name, cropped_contour)
    
cv2.destroyAllWindows()

output_shape_number_1.jpg
output_shape_number_2.jpg
output_shape_number_3.jpg
output_shape_number_4.jpg


## Approximating Contours and Convex Hull

cv2.approxPolyDP(contour, Approximation Accuracy, Closed)

- contour: is the contour we wish to approximate
- approximation accuracy: important parameter determining the accuracy of the approximation. Smaller value gives precise approximations, large values give more generic approximations. A good rule is less than 5% of the contour perimeter
- closed: a boolean value that states whether the approximate contour should be open or closed, closed is True in most cases    



In [4]:
import numpy as np
import cv2

# Load image and keep a copy

image = cv2.imread('./images/house.jpg')
orig_image = image.copy()
cv2.imshow('Original Image', orig_image)
cv2.waitKey()

# Grayscale and binarize
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

# Find contours 
contours, heirarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# Iterate through each contour and compute the bounding rectangle

for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    cv2.rectangle(orig_image, (x,y), (x+w, y+h), (0,0,255), 2)
    cv2.imshow('Bounding Rectangle', orig_image)
    
cv2.waitKey()

# Iterate through each contour and compute the approx contour

for c in contours:
    # Calculate accuracy as a percent of the contour perimeter
    accuracy = 0.03 * cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, accuracy, True)
    cv2.drawContours(image, [approx], 0, (0,255,0), 2)
    cv2.imshow('Approx Poly DP', image)
    
cv2.waitKey()
cv2.destroyAllWindows()

## Convex Hull

It is defined as the polygon of the minimum area that perfectly covers all the edges of the image


In [6]:
import numpy as np
import cv2

image = cv2.imread('./images/hand.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2.imshow('Original Image', image)
cv2.waitKey()

# Binarize the image
ret, thresh = cv2.threshold(gray, 176, 255, 0)

# Find contours
contours, heirarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# Sort Contours by area and then remove the largest frame contour
n = len(contours) - 1
contours = sorted(contours, key = cv2.contourArea, reverse = False)[:n]

# Iterate through contours and draw the convex hull

for c in contours:
    hull = cv2.convexHull(c)
    cv2.drawContours(image, [hull], 0, (0,255,0), 2)
    cv2.imshow('Convex Hull', image)
    
cv2.waitKey()
cv2.destroyAllWindows()

### Shape Matching
cv2.matchShapes(contour template, contour, method, method parameter)

Output - match value (lower values means a closer match)

- contour template: this is our reference contour that we're trying to find in the new image
- contour = the individual contour we are checking against
- method : type of contour matching 1, 2 or 3
- method parameter: leave alone as 0.0 (no fully utilized in Python in OpenCV)

In [3]:
import cv2
import numpy as np

# Load the shape template or reference image
template = cv2.imread('images/4star.jpg', 0)
cv2.imshow('Template', template)
cv2.waitKey()

# Load the target image with the shapes we're trying to match
target = cv2.imread('images/shapestomatch.jpg')
target_gray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)

# Threshold both images first before using cv2.findContours
ret, thresh1 = cv2.threshold(template, 127, 255, 0)
ret, thresh2 = cv2.threshold(template, 127, 255, 0)

contours, heirarchy = cv2.findContours(thresh1, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

# We need to sort the contours by area so that we can remove the largest 
# contour which is the image outline (we have to do this if the image
# background is not black, i.e., white in this case)

sorted_contours = sorted(contours, key = cv2.contourArea, reverse = True)

# We extract the second largest contour which will be our template contour
template_contour = contours[1]

# Extract contours from the second target image
contours, heirarchy = cv2.findContours(thresh2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

for c in contours:
    # Iterate through each contour in the target image and
    # use cv2.matchShapes to compare the contour shapes
    match = cv2.matchShapes(template_contour, c, 3, 0.0)
    print(match)
    
    # If the match value is less than 0.15 we
    if match < 0.15:
        closest_contour = c
    else:
        closest_contour = []
        
cv2.drawContours(target, [closest_contour], -1, (0,255,0), 3)
cv2.imshow('Output', target)
cv2.waitKey()
cv2.destroyAllWindows()

0.1313024004072794
0.0


http://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html

## Object detection: Template matching with OpenCV

## Gradient and Edge Detection

In [4]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    
    laplacian = cv2.Laplacian(frame, cv2.CV_64F)
    sobelx = cv2.Sobel(frame, cv2.CV_64F, 1, 0, ksize = 5)
    sobely = cv2.Sobel(frame, cv2.CV_64F, 0, 1, ksize = 5)
    
    edges = cv2.Canny(frame, 100, 100)
    cv2.imshow('original', frame)
    cv2.imshow('laplacian', laplacian)
    cv2.imshow('sobelx', sobelx)
    cv2.imshow('sobely', sobely)
    cv2.imshow('edges', edges)
    
    if(cv2.waitKey(1) == 13): # 13 is the Enter Key
        break

cv2.destroyAllWindows()
cap.release()

In [5]:
import cv2
import numpy as np

img = cv2.imread('./images/template-matching-main.jpg')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

template = cv2.imread('./images/template-matching.jpg', 0)

width, height = template.shape

result = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
threshold = 0.8
loc = np.where(result >= threshold)

for pt in zip(*loc[::-1]):
    cv2.rectangle(img, pt, (pt[0]+width, pt[1]+height), (0,255,255), 2)
    
cv2.imshow('detected', img)
cv2.waitKey()
cv2.destroyAllWindows()

### Mini Project 2 - Identiy Contours by Shape


In [7]:
import cv2
import numpy as np

# Load and then grayscale the image
image = cv2.imread('./images/someshapes.jpg')
gray  = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2.imshow('Original Image', image)
cv2.waitKey()

# Threshold the image
ret, thresh = cv2.threshold(gray, 127, 255, 1)

# Extract contours
contours = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

for cnt in contours: 
    
    # Get approximate polygons
    approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt,True), True)
    if(len(approx) == 3):
        shape_name = 'Triangle'
        cv2.drawContours(image, [cnt], 0, (0,255,0), -1)
        
        # Find contour center to place text at the center
        M = cv2.moments(cnt)
        cx = int(M['m10']/ M['m00'])
        cy = int(M['m01']/ M['m00'])
        cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
    
    elif len(approx) == 4:
        x, y, w, h = cv2.boundingRect(cnt)
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        
        # Check to see if 4 sided polygon is a square or a rectangle
        # cv2.boundingRect returns the top left and then width and
        if abs(w-h) <= 3:
            shape_name = 'Square'
            cv2.drawContours(image, [cnt], 0, (0,125,255), -1)
            cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
        else:
            shape_name = "Rectangle"
            
            # Find contour center to place text at the center
            cv2.drawContours(image, [cnt], 0, (0, 0, 255), -1)
            M = cv2.moments(cnt)
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
            
    elif len(approx) == 10:
        shape_name = 'Star'
        cv2.drawContours(image, [cnt], 0, (255, 255, 0), -1)
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
        
    elif len(approx) >= 15:
        shape_name = 'Circle'
        cv2.drawContours(image, [cnt], 0, (0,255,255), -1)
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
        
    cv2.imshow('Identifying shapes', image)
    cv2.waitKey()
    
cv2.destroyAllWindows()

TypeError: Expected Ptr<cv::UMat> for argument '%s'